In [ ]:
from ultralytics import YOLO
import cv2

Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\secre\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
# Carregar o modelo YOLO
model = YOLO("best.pt")

# Caminho do vídeo
video_path = "video.mp4"

# Abrir o vídeo
cap = cv2.VideoCapture(video_path)

valores = []
frame_atual = 0
intervalo = 4  # Pular a cada 4 frames

while cap.isOpened():
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_atual)
    ret, frame = cap.read()
    if not ret:
        break  # Sai do loop se o vídeo terminar

    # Realizar inferência no frame
    results = model(frame)

    # Iterar sobre os resultados
    for result in results:
        boxes = result.boxes  # Caixas delimitadoras
        classes = boxes.cls  # Classes detectadas (tensor)
        scores = boxes.conf  # Confiança das detecções (tensor)

        print(classes)
        # Converter os valores das classes para uma lista e armazenar
        # Verifica se o tensor está vazio
        if classes.numel() == 0:
            valores.append(4.0)  # Adiciona 4 se não houver detecção
        else:
            valores.extend(classes.tolist())  # Adiciona os valores normalmente

    frame_atual += intervalo

# Liberar recursos
cap.release()
cv2.destroyAllWindows()

In [ ]:
tamOriginal = [num for num in valores for _ in range(4)]

def contar_ocorrencias(arr, numero):
    return arr.count(numero)

def frames_para_tempo(frames):
    segundos = frames / 30
    minutos = int(segundos // 60)
    segundos_restantes = segundos % 60

    if(minutos == 0):
      return f"{segundos_restantes:.2f} seg"
    else:
      return f"{minutos} min {segundos_restantes:.2f} seg"

def calcular_porcentagem(arr, numero):
    total = len(arr)
    ocorrencias = arr.count(numero)
    porcentagem = round((ocorrencias / total) * 100, 3)
    return porcentagem

# EM PÉ:
frames_em_pe = contar_ocorrencias(tamOriginal, 1.0)
tempo_em_pe = frames_para_tempo(frames_em_pe)
porc_em_pe = calcular_porcentagem(tamOriginal, 1.0)

print('Tempo em pé:', tempo_em_pe, '(', porc_em_pe,'% do tempo total)')

# SENTADO:
frames_sentado = contar_ocorrencias(tamOriginal, 2.0)
tempo_sentado = frames_para_tempo(frames_sentado)
porc_sentado = calcular_porcentagem(tamOriginal, 2.0)

print('Tempo sentado:', tempo_sentado, '(', porc_sentado,'% do tempo total)')

# DEITADO:
frames_deitado = contar_ocorrencias(tamOriginal, 0.0)
tempo_deitado = frames_para_tempo(frames_deitado)
porc_deitado = calcular_porcentagem(tamOriginal, 0.0)

print('Tempo deitado:', tempo_deitado, '(', porc_deitado,'% do tempo total)')

# NAO:
frames_nao = contar_ocorrencias(tamOriginal, 4.0)
tempo_nao = frames_para_tempo(frames_nao)
porc_nao = calcular_porcentagem(tamOriginal, 4.0)

print('Tempo com usuário não detectado:', tempo_nao, '(', porc_nao,'% do tempo total)')

# TOTAL
duracao = frames_para_tempo(len(tamOriginal))

print('Duração total:', duracao)